In [1]:
import torch

In [2]:
import numpy as np
import csv
import random
import math
import matplotlib

import matplotlib.pyplot as plt

%matplotlib inline

#import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import copy
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as datasets
import torch.optim as optim
from torch import utils
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
use_cuda = torch.cuda.is_available()
CUDA_LAUNCH_BLOCKING=1
torch.cuda.set_device(0)
#torch.autograd.set_detect_anomaly(True)
print (use_cuda)

torch.backends.cudnn.enabled
import os

import sigopt
import time

True


/home/james/miniconda3/envs/james_mlmat/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.db import connect


from clease.tools import update_db
from clease import Concentration
from clease import CEBulk
from clease import Evaluate
from clease import NewStructures
from clease.calculator import Clease
from clease.calculator import attach_calculator

from mpl_toolkits.mplot3d import Axes3D  

import matplotlib.pyplot as plt
import numpy as np

import json
with open('agpd_dft_eci_final_submission.json') as json_file: 
    eci = json.load(json_file) 
    
conc = Concentration(basis_elements=[['Ag', 'Pd']])
settings = CEBulk(crystalstructure='fcc',
                   a=4.09,
                   size=[5,5,5],
                   concentration=conc,
                   db_name="agpd_dft_ref_final_submission.db",
                   max_cluster_size=4,
                   max_cluster_dia=[8.0, 6.5, 5.5])

atoms = settings.atoms.copy()
atoms = attach_calculator(settings, atoms=atoms, eci=eci)

gmeans = np.linspace(0,1,51)
gmeans_ten = torch.tensor(gmeans).cuda()

k_b = 0.00008617

def get_concentrations(lattices):
    lattices = ((lattices+1)/2).view(lattices.shape[0],-1)
    Ag_conc = torch.sum(lattices,dim=1)/(Dim*Dim*Dim)
    return Ag_conc

In [4]:

###### Based off Wu et Al. Solving Statistical Mechanics Using Variational Autoregressive Networks


def KL_loss(DBG,lattices,epoch,temp,field,num_temps,num_fields):
    lattices = lattices.detach()
    probs = DBG.get_sample_prob(lattices,temp,field,epoch).view(lattices.shape[0])
    with torch.no_grad():
        energies = DBG.get_energies(lattices).view(lattices.shape[0])
        energies_norm = energies.view(-1)/(temp.view(-1)*k_b)
       
        F = (energies_norm + probs).view(-1,1)
        F_new = F - (field.view(-1,1) * lattices.view(-1,DBG.Nz).sum(dim=1).view(-1,1))/(k_b*temp.view(-1,1))
        
        batch = int(lattices.shape[0]/(num_temps*num_fields))
        F_mean = F_new.view(-1,num_fields).view(num_temps,-1,num_fields)
        
        F_mean = F_mean.mean(dim=1).view(num_temps,1,num_fields).expand(num_temps,batch,num_fields)
        R = (F_new.view(-1) - F_mean.reshape(-1))/torch.abs(F_mean.reshape(-1))
        
    
    assert not R.requires_grad
    assert probs.requires_grad
    return torch.mean(R*probs)

In [5]:
temps_sim = [200,375,550,725,900]
field_sim = [-0.2,-0.1,0.0,0.1,0.2]
temp_temp = [0]
field_temp = [0]
coordinate_cutoff = 4.0
def train_model(model,optimizer,batch_size,epochs,temp_type,chem_type,temp_range):
    field_sim = [-0.2,-0.1,0.0,0.1,0.2]
    if temp_range == 'Large':
        temps_sim = [100,300,500,700,900]
    else:
        temps_sim = [200,375,550,725,900]
    epoch = 0
    temp = torch.zeros(len(temps_sim)*batch_size).cuda()
    field = torch.zeros(len(temps_sim)*batch_size).cuda()

    print_log_header()

    while epoch < epochs:
        if temp_type == 'Random Single':
            num_temps = 1
            t = temps_sim[0] + (temps_sim[-1]-temps_sim[0])*np.random.rand()
            for i in range(len(temps_sim)*batch_size):
                temp[i] = t
        elif temp_type == 'Random':
            t_fixed = []
            for i in range(5):
                t_temp = temps_sim[0] + (temps_sim[-1]-temps_sim[0])*np.random.rand()
                t_fixed.append(t_temp)
            num_temps = 5
            for i in range(len(temps_sim)*batch_size):
                temp[i] = t_fixed[int(i/batch_size)]
        else:
            num_temps = 5
            for i in range(len(temps_sim)*batch_size):
                temp[i] = temps_sim[int(i/batch_size)]
            
        if chem_type == 'Random Single':
            num_fields = 1
            f = -0.2 * 0.4*np.random.rand()
            for i in range(len(temps_sim)*batch_size):
                field[i] = f
        elif chem_type == 'Random':
            f_fixed = []
            for i in range(5):
                f_temp = -0.2 * 0.4*np.random.rand()
                f_fixed.append(f_temp)
            num_fields = 5
            for i in range(len(temps_sim)*batch_size):
                field[i] = f_fixed[int(i%(len(field_sim)))]
        else:
            num_fields = 5
            for i in range(len(temps_sim)*batch_size):
                field[i] = field_sim[int(i%(len(field_sim)))]
        
        
        epoch = epoch + 1

        lattices = model.forward(temp,field)
        
        
        kl_loss = KL_loss(model,lattices,epoch,temp,field,num_temps,num_fields)
        
        optimizer.zero_grad()
        kl_loss.backward()
        optimizer.step()
            
       
    return model

In [6]:
def print_log_header():
    print ('{:>8} {:>12}'\
       .format('epoch','train loss'))
    
def print_training_log(epoch, train_loss, test_loss=None):
    if test_loss is not None:
        print ('{:>8} {:>8} {:>12.4f} {:>12.4f}'\
                   .format(epoch, train_loss, test_loss))
        f.write('{:>8} {:>8} {:>12.4f} {:>12.4f}\n'\
                   .format(epoch, train_loss, test_loss))
    else:
        print ('{:>8} {:>8}'\
                   .format(epoch, train_loss))

In [7]:
def one_hot_to_sites(lattices,Nz):
    lattices = lattices.view(-1,Nz,2)
    lattices = torch.argmax(lattices,dim=2)
    lattices = 2*lattices - 1.0
    lattices = lattices.view(-1,Nz)
    return lattices.float()
    
def sites_to_one_hot(lattices):
    lattices = lattices.view(lattices.shape[0],-1)
    lattices = (0.5*(lattices + 1)).long()
    one_hot_lattices = torch.zeros(lattices.shape[0],lattices.shape[1],2).float().cuda()
    one_hot_lattices = one_hot_lattices.scatter_(2,lattices.view(lattices.shape[0],-1,1),1.0)
    return one_hot_lattices.detach()

In [8]:
def mask_weight(m,disp):
    num_sites = m.shape[1]
    num_components = 2
    mask = torch.zeros(m.shape)
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            if (disp == 0):
                if (j == 0 or j == 1 ):
                    mask[i,j] = 1.0
                elif ((j-2) < num_components*int(i/num_components) ):
                    mask[i,j] = 1.0
            if (disp == 1):
                if (j == 0 or j==1 ):
                    mask[i,j] = 1.0
                if ((j-2) < num_components*int(i/num_components) + num_components):
                    mask[i,j] = 1.0
    mask = mask.detach()
    m_masked = m*mask
    return m_masked

def get_zero_grad_hook(mask):
    def hook(grad):
        return grad * mask
    return hook

In [9]:
class autoreg_model(nn.Module):
    def __init__(self,Nz,assignments):
        super().__init__()
        self.Nz = Nz
        self.Dim = int(math.sqrt(Nz))
        self.assignments = assignments
        self.activation = activation_functions[assignments['activation']]
        self.lsoftmax = torch.nn.LogSoftmax(dim=2)
        self.softmax = torch.nn.Softmax(dim=2)
        self.num_layers = int(assignments['Layers'])
        
        self.shared_layer = nn.Linear(2*self.Nz+2,2*self.Nz,bias=False)
        with torch.no_grad():
            self.shared_layer.weight.copy_(np.sqrt(2)*mask_weight(self.shared_layer.weight,0))
        self.mask_1 = mask_weight(torch.ones_like(self.shared_layer.weight),0).cuda()
        
        self.shared_layer.weight.register_hook(get_zero_grad_hook(self.mask_1))
        
        
        self.shared_layer_2 = nn.Linear(2*self.Nz+2,2*self.Nz,bias=False)
        with torch.no_grad():
            self.shared_layer_2.weight.copy_(np.sqrt(2)*mask_weight(self.shared_layer_2.weight,1))
        self.mask_2 = mask_weight(torch.ones_like(self.shared_layer_2.weight),1).cuda()
        self.shared_layer_2.weight.register_hook(get_zero_grad_hook(self.mask_2))
        
        self.shared_layer_3 = nn.Linear(2*self.Nz+2,2*self.Nz,bias=False)
        with torch.no_grad():
            self.shared_layer_3.weight.copy_(np.sqrt(2)*mask_weight(self.shared_layer_3.weight,1))
        self.mask_3 = mask_weight(torch.ones_like(self.shared_layer_3.weight),1).cuda()
        self.shared_layer_3.weight.register_hook(get_zero_grad_hook(self.mask_3))
        
        self.shared_layer_4 = nn.Linear(2*self.Nz+2,2*self.Nz,bias=False)
        with torch.no_grad():
            self.shared_layer_4.weight.copy_(np.sqrt(2)*mask_weight(self.shared_layer_4.weight,1))
        self.mask_4 = mask_weight(torch.ones_like(self.shared_layer_4.weight),1).cuda()
        self.shared_layer_4.weight.register_hook(get_zero_grad_hook(self.mask_4))
        
    def get_sample_prob(self,lattices,temp,field,epoch):
        #print(self.shared_layer.weight)
        
        batch_size = temp.shape[0]
        samples = sites_to_one_hot(lattices).view(batch_size,-1)
        net_in = torch.cat((field.view(-1,1)*10,samples),dim=1)
        net_in = torch.cat((temp.view(-1,1)/1000,net_in),dim=1)
        conditional = self.activation(self.shared_layer(net_in))
        conditional = torch.cat((field.view(-1,1)*10,conditional),dim=1)
        conditional = torch.cat((temp.view(-1,1)/1000,conditional),dim=1)
        conditional = self.shared_layer_2(conditional)
        if self.num_layers > 2:
            conditional = self.activation(conditional)
            conditional = torch.cat((field.view(-1,1)*10,conditional),dim=1)
            conditional = torch.cat((temp.view(-1,1)/1000,conditional),dim=1)
            conditional = self.shared_layer_3(conditional)
            
        if self.num_layers > 3:
            conditional = self.activation(conditional)
            conditional = torch.cat((field.view(-1,1)*10,conditional),dim=1)
            conditional = torch.cat((temp.view(-1,1)/1000,conditional),dim=1)
            conditional = self.shared_layer_4(conditional)
        
        
        
        
        conditional = conditional.view(-1,self.Nz,2)
        Probs = self.lsoftmax(conditional)
        samples = samples.detach()
        assert Probs.requires_grad == True
        site_prob = Probs*(samples.view(-1,self.Nz,2))
        probs = torch.sum(torch.sum(site_prob,dim=2),dim=1)
        return probs
        
    def forward(self,temp,field):
        
        batch_size = temp.shape[0]
        temp = temp.view(-1,1)
        field = field.view(-1,1)
        lattices = torch.zeros(batch_size,2*self.Nz).cuda()
        for site in range(self.Nz):
            net_in = torch.cat((field*10,lattices),dim=1)
            net_in = torch.cat((temp/1000,net_in),dim=1)
            conditional = self.activation(self.shared_layer(net_in))
            conditional = torch.cat((field*10,conditional),dim=1)
            conditional = torch.cat((temp/1000,conditional),dim=1)
            conditional = self.shared_layer_2(conditional)
            if self.num_layers > 2:
                conditional = self.activation(conditional)
                conditional = torch.cat((field.view(-1,1)*10,conditional),dim=1)
                conditional = torch.cat((temp.view(-1,1)/1000,conditional),dim=1)
                conditional = self.shared_layer_3(conditional)
            
            if self.num_layers > 3:
                conditional = self.activation(conditional)
                conditional = torch.cat((field.view(-1,1)*10,conditional),dim=1)
                conditional = torch.cat((temp.view(-1,1)/1000,conditional),dim=1)
                conditional = self.shared_layer_4(conditional)
            
            conditional = conditional.view(-1,self.Nz,2)
            Probs = self.softmax(conditional)
            To_Sample = Probs[:,site,:]
            sample = torch.multinomial(To_Sample,1).view(-1,1)
            lattices = lattices.view(-1,self.Nz,2)
            lattices[:,site,:] = lattices[:,site,:].scatter_(1,sample,1)
            lattices = lattices.view(-1,2*self.Nz)
        #print(lattices.requires_grad)
        lattices = one_hot_to_sites(lattices,self.Nz)
        return lattices

def create_model(Nz,assignments):
    class Discrete_Boltzmann_Generator(nn.Module):
        def __init__(self,Nz,assignments):
            super().__init__()
            self.Nz = Nz
            self.Dim = int(math.sqrt(Nz))
            self.assignment = assignments
            self.Model = autoreg_model(Nz,assignments)
            atoms.numbers = np.ones((self.Nz))*46
            self.U_pd = atoms.get_potential_energy()
            atoms.numbers = np.ones((self.Nz))*47
            self.U_ag = atoms.get_potential_energy()
            
        def forward(self,temp,field):
            return self.Model.forward(temp,field)
            
        def get_energies(self,lattices):
            lattices = lattices.view(lattices.shape[0],-1)
            energies = torch.zeros(lattices.shape[0]).cuda()
            for lattice_num in range(lattices.shape[0]):
                energies[lattice_num] = self.get_cluster_energy(lattices[lattice_num,:])
            return energies
        
        def get_cluster_energy(self,lattice):
            atoms.numbers = (((lattice + 1)/2)*1 + 46).int().cpu().detach().numpy()
            energy_t = atoms.get_potential_energy()
            return energy_t
        
        def get_sample_prob(self,sample,temp,field,epoch):
            return self.Model.get_sample_prob(sample,temp,field,epoch)
        
    model = Discrete_Boltzmann_Generator(Nz,assignments).cuda()
    temp_range = assignments['temp_range']
    batch_size = int(assignments['batch_size'])
    optimizer = optimizers[assignments['optimizer']](model.parameters(), lr=10**assignments['log_learning_rate'])
    epochs = assignments['epochs']
    temp_type = assignments['temp_type']
    chem_type = assignments['Chem_type']
    model = train_model(model,optimizer,batch_size,epochs,temp_type,chem_type,temp_range)
    
    return model

In [10]:
activation_functions = {
    'relu': nn.ReLU(),
    'sigmoid': nn.Sigmoid(),
    'tanh': nn.Tanh(),
}

optimizers = {
    'gradient_descent': optim.SGD,
    'rmsprop': optim.RMSprop,
    'adam': optim.Adam,
}


Dim = 5

In [11]:
def get_Ness(model,lattices,tmp,temp_b,field_b):
    size = lattices.shape[0]
    iters = int(size/1000)
    lattices = lattices.view(-1,iters,125)
    for i in range(iters):
        energies = (model.get_energies(lattices[:,i,:])/(tmp*k_b)).view(-1)
        prob = model.get_sample_prob(lattices[:,i,:],temp_b,field_b,1000).view(-1)
        temp_pot = -(field_b.view(-1)*lattices[:,i,:].sum(dim=1).view(-1))/(tmp*k_b)
        temp_pot = energies+prob+temp_pot
        if i ==0:
            pot = temp_pot
        else:
            pot = torch.cat((pot,temp_pot),dim=0)
    log_rw = -pot
    log_rw = log_rw - log_rw.mean()
    log_rw = log_rw.view(-1)
    Top_term = 2*torch.logsumexp(log_rw,dim=0)
    Bottom_term = torch.logsumexp(2*log_rw,dim=0)
    Log_Ness = Top_term - Bottom_term
    return torch.exp(Log_Ness)

In [49]:
from sigopt import Connection
import socket

#conn = Connection(client_token="RAFA GB Client Token")
experiment_id = 357555
assignments = conn.experiments(experiment_id).best_assignments().fetch().data[0].assignments


NameError: name 'conn' is not defined

In [12]:
assignments = {
  "Chem_type": "Fixed",
  "Layers": "2",
  "activation": "tanh",
  "batch_size": "100",
  "epochs": 13285.14463955117,
  "log_learning_rate": -2.6845144509247336,
  "optimizer": "adam",
  "temp_range": "Small",
  "temp_type": "Random"
}

In [ ]:
model = create_model(125,assignments)

   epoch   train loss


In [ ]:
print(model)

In [ ]:
torch.save(model.state_dict(), "AgPdCE_125_final_submission")

In [49]:
def examine_phase(model,lattices,tmp,temp_b,field_b,Ag_conc):
    #print(lattices)
    prob = model.get_sample_prob(lattices,temp_b,field_b,1000).view(-1)
    with torch.no_grad():
        #print('examine_time')
        #en_st = time.time()
        energies = (model.get_energies(lattices)/(tmp*k_b)).view(-1)
        #en_end = time.time()
        #print(en_end-en_st)
        
        pot = -(field_b.view(-1)*lattices.sum(dim=1).view(-1))/(tmp*k_b)
        pot = energies+prob+pot
        log_rw = -pot
        
        log_rw_ns = log_rw - log_rw.mean()
        log_rw_ns = log_rw_ns.view(-1)
        Top_term = 2*torch.logsumexp(log_rw_ns,dim=0)
        Bottom_term = torch.logsumexp(2*log_rw_ns,dim=0)
        Log_Ness = Top_term - Bottom_term
    
        stored = log_rw.max()
        log_rw_corr = log_rw - stored
        rw = torch.exp(log_rw_corr)
        rw_norm = rw/(rw.sum())
        est_part = torch.exp(log_rw_corr).mean()
        log_z = torch.log(est_part)+stored
        #curr_pot = -(k_b*tmp)*(log_z).cpu().detach().numpy().item()
        conc_mean = (Ag_conc.view(-1)*rw_norm.view(-1)).sum().cpu().detach().numpy().item()

        gp = -k_b*tmp*log_z
        
    return conc_mean,gp,Log_Ness

In [62]:
def get_conc_at_temp(temp):
    
    chemical_pots = np.linspace(-0.2,0.2,81)
    concs = np.linspace(-.2,.2,81)
    concs_rw = np.linspace(-.2,.2,81)
    gps = np.linspace(-.2,.2,81)
    num_samps = np.linspace(-.2,.2,81)
    temps = torch.zeros(5000).cuda()
    counter = 0
    for chpot in chemical_pots:
        temps = torch.zeros(5000).cuda()
        fields = torch.zeros(5000).cuda()
        for i in range(5000):
            temps[i] = temp
            fields[i] = chpot
            
        lattices = model.forward(temps,fields)
        concs_iter = get_concentrations(lattices).view(-1)
        
        conc_mean,gp,num_samp = examine_phase(model,lattices,temp,temps,fields,concs_iter)
        gps[counter] = gp.cpu().detach().numpy()
        concs_rw[counter] = conc_mean
        num_samps[counter] = num_samp
        counter = counter + 1
        
    return chemical_pots,concs_rw,gps,num_samps

In [ ]:
AgPd_data_concs = np.zeros((15,81))
AgPd_data_gp = np.zeros((15,81))
AgPd_data_num_samp = np.zeros((15,81))
temps_study = np.linspace(200,900,15)
print(temps_study)
print()

pos = 0
for t in temps_study:
    chemical_pots,concs_rw,gp,num_samp = get_conc_at_temp(t)
    AgPd_data_concs[pos,:] = concs_rw
    AgPd_data_gp[pos,:] = gp
    AgPd_data_num_samp[pos,:] = num_samp
    pos+=1
    print(pos)

[200. 250. 300. 350. 400. 450. 500. 550. 600. 650. 700. 750. 800. 850.
 900.]

1


In [ ]:
np.savetxt("AgPd_AR_125_concs_81_final_submission_3.csv", AgPd_data_concs, delimiter=",")
np.savetxt("AgPd_AR_125_samps_81_final_submission_3.csv", AgPd_data_num_samp, delimiter=",")

In [ ]:
def get_Ness(model,lattices,tmp,temp_b,field_b):
    size = lattices.shape[0]
    iters = int(size/1000)
    lattices = lattices.view(-1,iters,125)
    for i in range(iters):
        energies = (model.get_energies(lattices[:,i,:])/(k_b*tmp)).view(-1)
        prob = model.get_sample_prob(lattices[:,i,:],temp_b,field_b,1000).view(-1)
        temp_pot = -(field_b.view(-1)*lattices[:,i,:].sum(dim=1).view(-1))/(k_b*tmp)
        temp_pot = energies+prob+temp_pot
        if i ==0:
            pot = temp_pot
        else:
            pot = torch.cat((pot,temp_pot),dim=0)
    log_rw = -pot
    log_rw = log_rw - log_rw.mean()
    log_rw = log_rw.view(-1)
    Top_term = 2*torch.logsumexp(log_rw,dim=0)
    Bottom_term = torch.logsumexp(2*log_rw,dim=0)
    Log_Ness = Top_term - Bottom_term
    return torch.exp(Log_Ness)

In [ ]:
chem_p = np.linspace(-0.2,0.2,41)
temps = np.linspace(200,900,21)
num = 10000
temps_f = torch.zeros(1000).cuda()
ch_p_f = torch.zeros(1000).cuda()
Ness = np.zeros((temps.shape[0],chem_p.shape[0]))
for i in range(temps.shape[0]):
    for j in range(chem_p.shape[0]):
        for m in range(10):
            for k in range(1000):
                temps_f[k] = temps[i]
                ch_p_f[k] = chem_p[j]
            temp_lattices = model.forward(temps_f,ch_p_f)
            if m ==0:
                lattices = temp_lattices
            else:
                lattices = torch.cat((lattices,temp_lattices),dim=0)
        N_temp = get_Ness(model,lattices,temps[i],temps_f,ch_p_f)
        Ness[i,j] = N_temp/num
        print("done")

In [ ]:
np.save("AgPd_125_NESS_data_final_submission", Ness)

In [ ]:
print(Ness)

In [ ]:
log_ness = np.log10(Ness)

In [ ]:
print(np.max(log_ness))
print(np.min(log_ness))

In [ ]:
all_chem_p = []
all_temps = []
for i in range(temps.shape[0]):
    for j in range(chem_p.shape[0]):
        all_temps.append(temps[i])
        all_chem_p.append(2*chem_p[j])
        
plt.hist2d(all_chem_p,all_temps,bins = (41,21), weights = np.log10(Ness).reshape(-1),cmap='inferno',vmin=-4.0,vmax=0.0)
plt.rcParams["figure.figsize"] = (7,5)
plt.xlabel('Chemical Potential (eV)',fontsize=16)
plt.ylabel('Temperature (K)',fontsize=16)
plt.xticks(fontsize = 16,fontname = "Arial") 
plt.yticks(fontsize = 16,fontname = "Arial") 
plt.yticks([200,400,600,800])
plt.xticks([-0.4,-0.2,0.0,0.2,0.4])

cb = plt.colorbar()

imaxes = plt.gca()
plt.axes(cb.ax)
plt.yticks(fontsize=16,fontname = "Arial")
cb.set_ticks([-4.0,-3.0,-2.0,-1.0,0.0])


plt.savefig("NESS_AgPd_125_Final_Submission.pdf",bbox_inches='tight')